In [ ]:
################################################################################
# INSTALL CONDA ON GOOGLE COLAB
################################################################################
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
#Mounting google drive for workspace
from google.colab import drive 
drive.mount('/content/gdrive')

In [ ]:
#Change directory to working folder (change address to your own folder's address)
%cd /content/gdrive/My Drive/Colab Notebooks/BE Thesis/Prediction

Paste the Keras-LSTM-Trajectory-Prediction folder in the working folder

In [ ]:
%cd Keras-LSTM-Trajectory-Prediction

In [ ]:
#Installing dependencies
!conda env update -f environment.yml 

In [ ]:
pip install --upgrade progressbar2

In [ ]:
#Test with a random set of trajectories (No need to execute)
!python load.py

# **Workflow of the model training:**

1. Extract raw object trajectories obtained from a previous Detection + Tracking processing pipeline.
In order to train the trajectory forcasting model, you need several .csv files containing raw trajectories obtained from a previous Detection + Tracking processing pipeline, a D+T pipeline for short. The D+T pipeline is in charge of supplying raw D+T trajectories packed in a .csv file with an specific output format. All of the previously described .csv files must be in a folder for further use by dataset_creator utility tool.



2. Pack all trajectories in a convenient raw_paths.h5 file
Once you have all D+T raw trajectories on an user-defined cache_folder, use dataset_creator utility tool to pack all raw trajectories into a raw_paths.h5 file located at output_folder = full/path/to/raw_paths.h5: .
The resulting raw_paths.h5 will contain several object trajectories for furter steps. cache_folder is where all the raw CSV files are given as inputs. output_folder is where the create .h5 file will be stored.

In [ ]:
%cd dataset-creator/

*Usage notes for dataset-creator.py:*


```
dataset-creator [-h] [-a] input_directory output

positional arguments:

	input_directory  Path of a directory with CSV files to extract tracks.

	output           Path of the output training dataset file with .hdf5 extension.

optional arguments:

	-h, --help       show this help message and exit
  
	-a, --append     Flag to indicate that an existing HDF5 file can be used and new datasets should be appended to it.
```



In [ ]:
!python dataset-creator.py /content/gdrive/My\ Drive/Colab\ Notebooks/BE\ Thesis/Prediction/Keras-LSTM-Trajectory-Prediction/cache_folder /content/gdrive/My\ Drive/Colab\ Notebooks/BE\ Thesis/Prediction/Keras-LSTM-Trajectory-Prediction/output_folder/raw_paths.hdf5

3. Transform and group all raw paths from raw_paths.h5 into an train_test.h5 dataset
All raw paths from raw_paths.h5 will be used for dataset_transformer utility tool in order to generate a train-test dataset for further training and inference tasks. In addition, internal processing, filtering and polynomial trajectory smoothing can be configured by the user using the associated config_transformer.json file. The tool receives a raw_dataset = full/path/to/raw_paths.h5 path, an it will generate a train-test dataset in .h5 format at location out_dataset = full/path/to/train-test.h5 using the config_transformer.json file avalaible at location config = full/path/to/config_transformer.json:


In [ ]:
%cd /content/gdrive/My Drive/Colab\ Notebooks/BE\ Thesis/Prediction/Keras-LSTM-Trajectory-Prediction/dataset_transformer

*Usage notes for dataset_transformer.py:*



```
dataset_transformer.py [-h] raw_dataset out_dataset configuration_file

positional arguments:

  raw_dataset  Path to input raw dataset in .h5 format

  out_dataset  Path to output train/test dataset in .h5 format

  config       Path to configuration file used for dataset transformer tool.

optional arguments:

  -h, --help            show this help message and exit
  
  --norm_target_data    Normalizes or smooths target trajectory data using nth-degree polynomial regression. nth-degree parameter specified in config.json file
```



In [ ]:
!python dataset_transformer.py /content/gdrive/My\ Drive/Colab\ Notebooks/BE\ Thesis/Prediction/Keras-LSTM-Trajectory-Prediction/output_folder/raw_paths.hdf5 /content/gdrive/My\ Drive/Colab\ Notebooks/BE\ Thesis/Prediction/Keras-LSTM-Trajectory-Prediction/train_test/train-test.hdf5 /content/gdrive/My\ Drive/Colab\ Notebooks/BE\ Thesis/Prediction/Keras-LSTM-Trajectory-Prediction/dataset_transformer/config_transformer.json


4. Train and evaluate a Path Prediction LSTM-based neural model
By using the LSTM_trainer.py script, the previously generated train-test.h5 dataset, and an user configuration file config_lstm.json you can train your own custom LSTM-based Trajectory prediction model.
In addition, in order to evaluate the performance of the trained models, use the evaluate_lstm.py script using also the previously generated train-test.h5 dataset, and the same user configuration file config_lstm.json used for training. The evaluate_lstm.py script contains code use for load datasets, load trained models and making inference with them.



In [ ]:
%cd /content/gdrive/My Drive/Colab\ Notebooks/BE\ Thesis/Prediction/Keras-LSTM-Trajectory-Prediction/Train_Test_Scripts

# **Training of the proposed model**

*Usage notes for LSTM_Trainer.py* : 



```
LSTM_trainer.py [-h] dataset  output  config

Positional arguments:

  dataset -     Path to input train-test dataset in .hdf5 format
  
  output  -   Path to the output directory where resulting models, graphs and history results are saved.

  config   -  Path to configuration file used for training.

optional arguments:

  -h, --help    -    show this help message and exit

  --use_checkpoint - Load and use a previously trained LSTM model to restart training from that file. A new and restored model will be created, preserving original input model.
```


In [ ]:
!python LSTM_trainer.py /content/gdrive/My\ Drive/Colab\ Notebooks/BE\ Thesis/Prediction/Keras-LSTM-Trajectory-Prediction/train_test/train-test.hdf5 /content/gdrive/My\ Drive/Colab\ Notebooks/BE\ Thesis/Prediction/Keras-LSTM-Trajectory-Prediction/model_output config_lstm.json

# **Evaluation of the trained model**


*Usage notes for evaluate_lstm.py:*



```
evaluate_lstm.py [-h] dataset output configuration_file

positional arguments:

  dataset   -  Path to input test dataset in .hdf5 format

  output    -  Path to the output directory where resulting training graphs and 
  prediction results (model folder) are computed.

  config    -  Path to configuration file used for testing.

optional arguments:
  -h, --help  show this help message and exit
```


In [ ]:
%cd /content/gdrive/My Drive/Colab\ Notebooks/BE\ Thesis/Prediction/Keras-LSTM-Trajectory-Prediction/Train_Test_Scripts

In [ ]:
!python evaluate_lstm.py  /content/gdrive/My\ Drive/Colab\ Notebooks/BE\ Thesis/Prediction/Keras-LSTM-Trajectory-Prediction/train_test/train-test.hdf5 /content/gdrive/My\ Drive/Colab\ Notebooks/BE\ Thesis/Prediction/Keras-LSTM-Trajectory-Prediction/model_output/ARCH--15i_30o_10sld_norm3_vill_cascade2-_Data-_final_concat512___bs-512_lr-0.01_loss-mae_opt-Ranger_BD-True_BDmrg-concat_amsG-False_DP-False_sw-0.9_sync-1_act-selu_minLR-1e-05_ptc-10_ep-100 config_lstm.json